### Q&A Bot using RAG (Retrieval Augmented Generation)

This notebook builds a simple Q&A bot:
1. Loads local text files
2. Splits them into chunks
3. Creates vector embeddings using OpenAI
4. Stores them in FAISS vector database
5. Retrieves relevant chunks for a query
6. Uses GPT model to generate answers

In [6]:
# Imports
from openai import OpenAI
import faiss
import numpy as np
import glob, os, textwrap

#### 1. Load Documents

In [7]:
def load_documents(path="data/*.txt"):
    docs = []
    for file in glob.glob(path):
        with open(file, 'r', encoding='utf-8') as f:
            text = f.read()
            for chunk in textwrap.wrap(text, width=500):
                docs.append(chunk)
    return docs


documents = load_documents()
print(f"Loaded {len(documents)} text chunks.")

Loaded 1 text chunks.


#### 2. Create Embeddings & Vector Index

In [9]:
client = OpenAI(api_key="<YOUR_API_KEY>")
embeddings = []
for doc in documents:
    e = client.embeddings.create(model="text-embedding-3-small", input=doc)
    embeddings.append(e.data[0].embedding)

embeddings = np.array(embeddings).astype("float32")
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

#### 3. Search Relevant Chunks

In [10]:
def search(query, k=3):
    e = client.embeddings.create(model="text-embedding-3-small", input=query)
    q = np.array(e.data[0].embedding).astype("float32").reshape(1, -1)
    D, I = index.search(q, k)
    return [documents[i] for i in I[0]]

#### 4. Ask GPT

In [11]:
def ask_gpt(question):
    context = "\n".join(search(question))
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Answer based only on context."},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {question}"}
        ]
    )
    return response.choices[0].message.content

#### 5. Test Q&A Bot

In [12]:
print("Answer:", ask_gpt("What is this document about?"))

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}